# 标准库中的装饰器

**Python内置了3个用于装饰方法的函数：property、classmethod、staticmethod**      
**functools里面也有几个很有用的装饰器：**      
**functools.wraps、functools.lru_cache、functools.singledispatch**

## 1. functools.lru_cache做备忘

**```functools.lru_cache``` 实现了备忘功能，把耗时的函数结果保存起来，避免传入相同的参数时重复计算**     
* *lru -> least recently used*，缓存不会无限制增长，一段时间不用的缓存会被清除掉

In [1]:
import time
import functools

In [2]:
def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [3]:
@clock
def fabonacci(n):
    return n if n < 2 else fabonacci(n-2) + fabonacci(n-1)

In [4]:
fabonacci(6)

[0.00000032s] fabonacci(0) -> 0
[0.00000032s] fabonacci(1) -> 1
[0.00005229s] fabonacci(2) -> 1
[0.00000032s] fabonacci(1) -> 1
[0.00000032s] fabonacci(0) -> 0
[0.00000032s] fabonacci(1) -> 1
[0.00003368s] fabonacci(2) -> 1
[0.00005293s] fabonacci(3) -> 2
[0.00012640s] fabonacci(4) -> 3
[0.00000000s] fabonacci(1) -> 1
[0.00000000s] fabonacci(0) -> 0
[0.00000064s] fabonacci(1) -> 1
[0.00025985s] fabonacci(2) -> 1
[0.00027974s] fabonacci(3) -> 2
[0.00000032s] fabonacci(0) -> 0
[0.00000000s] fabonacci(1) -> 1
[0.00003914s] fabonacci(2) -> 1
[0.00000032s] fabonacci(1) -> 1
[0.00000032s] fabonacci(0) -> 0
[0.00000000s] fabonacci(1) -> 1
[0.00001700s] fabonacci(2) -> 1
[0.00003240s] fabonacci(3) -> 2
[0.00008854s] fabonacci(4) -> 3
[0.00038593s] fabonacci(5) -> 5
[0.00052964s] fabonacci(6) -> 8


8

**上面这种计算斐波那契数列的方法明显有很多重复计算，浪费很多时间，使用 ```lru_cache``` 会显著改善性能，优化递归算法**

In [5]:
@functools.lru_cache()   # 有括号是因为它可以接受配置参数
@clock                   # 装饰器叠放，@lru_cache()应用到 @clock 返回的函数上
def fabonacci(n):
    return n if n < 2 else fabonacci(n-2) + fabonacci(n-1)

In [6]:
fabonacci(6)

[0.00000032s] fabonacci(0) -> 0
[0.00000032s] fabonacci(1) -> 1
[0.00006384s] fabonacci(2) -> 1
[0.00000032s] fabonacci(3) -> 2
[0.00007988s] fabonacci(4) -> 3
[0.00000096s] fabonacci(5) -> 5
[0.00016778s] fabonacci(6) -> 8


8

In [7]:
fabonacci(30)

[0.00000064s] fabonacci(7) -> 13
[0.00002919s] fabonacci(8) -> 21
[0.00000064s] fabonacci(9) -> 34
[0.00015334s] fabonacci(10) -> 55
[0.00000064s] fabonacci(11) -> 89
[0.00018222s] fabonacci(12) -> 144
[0.00000032s] fabonacci(13) -> 233
[0.00020082s] fabonacci(14) -> 377
[0.00000032s] fabonacci(15) -> 610
[0.00021943s] fabonacci(16) -> 987
[0.00000032s] fabonacci(17) -> 1597
[0.00023611s] fabonacci(18) -> 2584
[0.00000064s] fabonacci(19) -> 4181
[0.00042827s] fabonacci(20) -> 6765
[0.00000064s] fabonacci(21) -> 10946
[0.00044752s] fabonacci(22) -> 17711
[0.00000032s] fabonacci(23) -> 28657
[0.00046613s] fabonacci(24) -> 46368
[0.00000032s] fabonacci(25) -> 75025
[0.00048377s] fabonacci(26) -> 121393
[0.00000064s] fabonacci(27) -> 196418
[0.00050141s] fabonacci(28) -> 317811
[0.00000032s] fabonacci(29) -> 514229
[0.00051906s] fabonacci(30) -> 832040


832040

**```lru_cache```可以使用两个可选的配置参数：** ```lru_cache(maxsize=128, typed=False)```       
```maxsize```: 指定存储多少个调用的结果，为了得到最佳性能，应设置为2的幂       
```typed```: 如果设置成True，把不同的参数类型得到的结果分开保存         
**```lru_cache```使用字典存储结果，而且键根据调用时传入的定位参数和关键字参数创建，所以被它装饰的函数的所有参数必须是可散列的**

## 2. 单分派泛函数